In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objects as go

from dash.dependencies import Input, Output

In [2]:
import pandas as pd

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

In [4]:
dmc_df = pd.read_csv('../data/dmc.csv', index_col='number')
metrics = [
    'RGB euclidean',
    'RGB with gamma correction',
    'CIE76',
    'CIE94',
    'CIEDE2000',
    'CMC 1:1',
    'CMC 2:1'
]

In [21]:
def draw_graph():
    base_color = 'blue'
    compared_colors = ['red', 'green', 'yellow']

    data_axis = 0
    xlabels = ['11', '12', '13']
    ylabels = ['xD', ' ']

    data = [[3, 3, 3], [0, 1, 2]]

    fig = go.Figure(data=go.Heatmap(
        z=np.stack((np.arange(n), [n] * n), data_axis),
        x=xlabels,
        y=[ylabels[1], ylabels[0]],
        colorscale=compared_colors + [base_color],
        showscale=False,
        xgap=4
    ))
    
    return fig

In [22]:
app.layout = html.Div([dbc.Row([
    dbc.Col([
        html.Label('Base DMC color:'),
        dcc.Dropdown(
            id='drop_dmc',
            options=[{'label': code, 'value': code}
                     for code in dmc_df.index],
            placeholder='Select DMC color',
            searchable=True,
            clearable=False,
            style={'marginBottom': '1.5em'},
        ),
        html.Label('Color comparing algorithm:'),
        dcc.Dropdown(
            id='drop_metric',
            options=[{'label': m, 'value': m} for m in metrics],
            placeholder='Select algorithm',
            searchable=False,
            value='CIEDE2000',
            clearable=False,
            style={'marginBottom': '1.5em'},
        ),
        html.Label('Number of similar colors:'),
        html.Div(dcc.Slider(
            id='slider_colors',
            min=1,
            max=9,
            step=1,
            value=5,
            marks={n: str(n) for n in range(1, 10, 2)},
        ), style={'marginBottom': '1.5em'},),
        html.Div([
            dbc.Button(
                'Clear',
                color='secondary',
                id='button_clear',
                outline=True
            ),
            dbc.Button(
                'Find similar',
                color='primary',
                id='button_run'
            ),], style={'display': 'flex', 'justify-content': 'right'}),
    ], width=3),
    dbc.Col(html.Div([
        dcc.Graph(
            id='graph',
            figure=draw_graph()
        )
    ]), width=6),
])])

In [24]:
app.run_server(mode='inline', debug=True)